This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
 

In [1]:
figname = '../paper/fig_params'

In [2]:
%run 0_parameters.ipynb


This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
       
      
Using filename= ../data/2019-03-19
{'w': 28, 'minibatch_size': 100, 'train_batch_size': 50000, 'test_batch_size': 5000, 'noise_batch_size': 1000, 'mean': 0.1307, 'std': 0.3081, 'N_pic': 128, 'offset_std': 30, 'offset_max': 35, 'noise': 0.7, 'contrast': 0.8, 'sf_0': 0.2, 'B_sf': 0.3, 'N_theta': 6, 'N_azimuth': 16, 'N_eccentricity': 10, 'N_phase': 2, 'rho': 1.41, 'bias_deconv': True, 'p_dropout': 0.5, 'dim1': 1000, 'dim2': 1000, 'lr': 0.0001, 'do_adam': True, 'bn1_bn_momentum': 0.0, 'bn2_bn_momentum': 0.0, 'momentum': 0.1, 'epochs': 50, 'num_processes': 1, 'no_cuda': True, 'log_interval': 100, 'verbose': 1, 'filename': '../data/2019-03-19', 'seed': 2019, 'N_cv': 2}
Overwriting train.py
total 224
-rw-r--r--  1 laurentperrinet  staff   216B Mar 18 14:28 sf_0__0_141.npy
-rw-r--r--  1 laurentperrinet  staff   216B Mar 18 14:29 sf_0__0_100.

In [3]:
%%writefile params.py
import torch
from main import init, MetaML

opts = dict(filename='../data/2019-03-16', verbose=0, log_interval=0)
#opts = dict(filename='debug', verbose=0, log_interval=0)

print(50*'-')
print(' parameter scan')
print(50*'-')

args = init(**opts)
from where import Where as ML
from what import WhatNet
ml = ML(args)
if False:
    print(50*'-')
    print('Default parameters')
    print(50*'-')

    ml.train(path=args.filename)
    # ml.main(path=args.filename)

if True:
    args = init(**opts)
    mml = MetaML(args)
    if torch.cuda.is_available():
        mml.scan('no_cuda', [True, False])

    args = init(**opts)
    mml = MetaML(args)
    mml.scan('bias_deconv', [True, False])
   

for base in [2, 8] if not args.filename == '../data/debug' else [2]:
    print(50*'-')
    print(' base=', base)
    print(50*'-')

    print(50*'-')
    print(' parameter scan : data')
    print(50*'-')
    args = init(**opts)
    mml = MetaML(args, base=base)
    for parameter in ['sf_0', 'B_sf', 'offset_std', 'noise', 'contrast']:
        mml.parameter_scan(parameter)
        
    # TODO:  'N_theta': 6, 'N_azimuth': 16, 'N_eccentricity': 10, 'rho': 1.41,

    print(50*'-')
    print(' parameter scan : network')
    print(50*'-')
    args = init(**opts)
    mml = MetaML(args)
    for parameter in ['dim1',
                      'bn1_bn_momentum',
                      'dim2',
                      'bn2_bn_momentum',
                      'p_dropout']:
        mml.parameter_scan(parameter)

    print(' parameter scan : learning ')
    args = init(**opts)
    mml = MetaML(args, base=base)
    print(50*'-')
    print('Using SGD')
    print(50*'-')
    for parameter in ['lr', 'momentum', 'minibatch_size', 'epochs']:
        mml.parameter_scan(parameter)
    print(50*'-')
    print('Using ADAM')
    print(50*'-')
    args = init(**opts)
    args.do_adam = True
    mml = MetaML(args, tag='adam')
    for parameter in ['lr', 'momentum', 'minibatch_size', 'epochs']:
        mml.parameter_scan(parameter)



Overwriting params.py


In [4]:
from where import Where as ML

In [5]:
args.filename

'../data/2019-03-19'

In [6]:
%run params.py

--------------------------------------------------
 parameter scan
--------------------------------------------------
scanning over bias_deconv = [True, False]
For parameter bias_deconv = True ,  Accuracy=43.2% +/- 0.7%  in 249598.9 seconds
For parameter bias_deconv = False ,  Accuracy=42.7% +/- 0.5%  in 249434.0 seconds
--------------------------------------------------
 base= 2
--------------------------------------------------
--------------------------------------------------
 parameter scan : data
--------------------------------------------------
scanning over sf_0 = [0.1        0.11892071 0.14142136 0.16817928 0.2        0.23784142
 0.28284271 0.33635857 0.4       ]
For parameter sf_0 = 0.100 ,  Accuracy=34.7% +/- 0.6%  in 248961.2 seconds
For parameter sf_0 = 0.119 ,  Accuracy=35.7% +/- 0.5%  in 248985.7 seconds
For parameter sf_0 = 0.141 ,  Accuracy=37.4% +/- 0.5%  in 248932.7 seconds
For parameter sf_0 = 0.168 ,  Accuracy=40.1% +/- 0.7%  in 249173.5 seconds
For parameter sf_0

  0%|          | 0/50 [00:00<?, ?it/s]


Failed with error not enough values to unpack (expected 3, got 2)
Failed with error 1000
For parameter dim1 = 1189 ,   currently locked with  ../data/_tmp_scanning/dim1__1189.npy_lock
For parameter dim1 = 1414 ,  

  0%|          | 0/50 [00:00<?, ?it/s]


Failed with error not enough values to unpack (expected 3, got 2)
Failed with error 1414
For parameter dim1 = 1681 ,  

...

Failed with error not enough values to unpack (expected 3, got 2)
Failed with error 10
For parameter epochs = 17 ,   currently locked with  ../data/_tmp_scanning/epochs__17.npy_lock
For parameter epochs = 29 ,   currently locked with  ../data/_tmp_scanning/epochs__29.npy_lock
For parameter epochs = 50 ,   currently locked with  ../data/_tmp_scanning/epochs__50.npy_lock
For parameter epochs = 84 ,   currently locked with  ../data/_tmp_scanning/epochs__84.npy_lock
For parameter epochs = 141 ,  

  0%|          | 0/141 [00:00<?, ?it/s]


Failed with error not enough values to unpack (expected 3, got 2)
Failed with error 141
For parameter epochs = 237 ,   currently locked with  ../data/_tmp_scanning/epochs__237.npy_lock
For parameter epochs = 400 ,  

  0%|          | 0/400 [00:00<?, ?it/s]

Failed with error not enough values to unpack (expected 3, got 2)
Failed with error 400
--------------------------------------------------
Using ADAM
--------------------------------------------------
scanning over lr = [5.00000000e-05 5.94603558e-05 7.07106781e-05 8.40896415e-05
 1.00000000e-04 1.18920712e-04 1.41421356e-04 1.68179283e-04
 2.00000000e-04]
For parameter lr = 0.000 ,   currently locked with  ../data/_tmp_scanning/lr_adam_0_000.npy_lock
For parameter lr = 0.000 ,   currently locked with  ../data/_tmp_scanning/lr_adam_0_000.npy_lock
For parameter lr = 0.000 ,   currently locked with  ../data/_tmp_scanning/lr_adam_0_000.npy_lock
For parameter lr = 0.000 ,   currently locked with  ../data/_tmp_scanning/lr_adam_0_000.npy_lock
For parameter lr = 0.000 ,   currently locked with  ../data/_tmp_scanning/lr_adam_0_000.npy_lock
For parameter lr = 0.000 ,   currently locked with  ../data/_tmp_scanning/lr_adam_0_000.npy_lock
For parameter lr = 0.000 ,   currently locked with  ../data

# summary


In [7]:
%ls ../data/_tmp_scanning

B_sf__0_037.npy_lock              minibatch_size__168.npy_lock
B_sf__0_063.npy_lock              minibatch_size__200.npy_lock
B_sf__0_106.npy_lock              minibatch_size__21.npy_lock
B_sf__0_150.npy                   minibatch_size__282.npy_lock
B_sf__0_178.npy                   minibatch_size__35.npy_lock
B_sf__0_212.npy                   minibatch_size__475.npy_lock
B_sf__0_252.npy                   minibatch_size__50.npy_lock
B_sf__0_300.npy                   minibatch_size__59.npy_lock
B_sf__0_357.npy                   minibatch_size__70.npy_lock
B_sf__0_424.npy                   minibatch_size__800.npy_lock
B_sf__0_505.npy                   minibatch_size__84.npy_lock
B_sf__0_600.npy                   minibatch_size_adam_100.npy_lock
B_sf__0_849.npy_lock              minibatch_size_adam_118.npy_lock
B_sf__1_427.npy_lock              minibatch_size_adam_141.npy_lock
B_sf__2_400.npy_lock              minibatch_size_adam_168.npy_lock
bias_deconv__False.npy            minibatch_s

# post-processing

In [8]:
!ls  -l {figname}*

ls: ../paper/fig_params*: No such file or directory
